# xmlObject and Rosetta Script in PyRosetta

@Author:Jian Huang

@E-mail: jian.huang@xtalpi.com

# 1. Introduction

在rosetta软件开始发展的初期，其功能是以单个单个的应用提交给用户使用的，例如我们经常使用的socre，relax等。作为开发者和设计者，当然不会满足于这些没办法定制自己独有设计过程的程序。rosetta基于不同用户有的不同需求，给定用户更高的自由度——直接使用比应用更底层的功能进行组装，形成自己的“应用”或protocol。

首先观察一下Rosetta Script的标准框架：
```
<ROSETTASCRIPTS>
    <SCOREFXNS>
    </SCOREFXNS>
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <SIMPLE_METRICS>
    </SIMPLE_METRICS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
    </MOVERS>
    <PROTOCOLS>
    </PROTOCOLS>
    <OUTPUT />
</ROSETTASCRIPTS>
```

在一个RS基本流程中，用户输入一个蛋白构象（pose），通过定义的movers改变构象，设置filters对改变后的构象进行评估，然后输出符合条件的构象。RS流程允许用户将rosetta的底层（movers，如pack，minimizer等）线性组合起来，形成自己独特的设计流程，极度强大。


***

# 2. RS基础
仔细观察RS框架，可以发现每一项都是通过```<name>  </name>```申明的，例如FILTER，可想而知，我们就应该在两个<>中间写入必要的内容进行操作了，而介于两种<>之间的空隙可以用来写comment：

```
<ROSETTASCRIPTS>
    <SCOREFXNS>
    </SCOREFXNS>
    
    这块地方介于两项<>的中间，可以用来写注释，不会被rosetta读入进去！
    
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <SIMPLE_METRICS>
    </SIMPLE_METRICS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
    
    写在这里的定义会被rosetta读入进去！
    
    </MOVERS>
    <PROTOCOLS>
    </PROTOCOLS>
    <OUTPUT />
</ROSETTASCRIPTS>
```

注意，下面两种写法等同：称之为一个tag

```
# 1
<SCOREFXNS>
</SCOREFXNS>

# 2
<SCOREFXNS/>（内容写在tag中间）
```

一般为了注重可读性，会使用缩进或空格的方式，将内层的层级凸显出来。这仅仅是为了script的可读性。当一个tag中存在选项列表的时候，列表中各个值应该以逗号分隔，且不应该含有空格。

```
<PackRotamers name="pack1" task_operations="task1,task2,task3" /> #This is allowed
<PackRotamers name="pack2" task_operations="task2, task2, task3" /> #This will be misinterpreted
```

在RS中的这些tag中，可以创建由tag决定类型的实例，例如在movers中，我们可以写很多种不同的mover，并且可以给他们取不同的名字：

```
<MOVERS>  #In this section, movers are defined.
          # We assume that task1, task2, and task3 were defined and given these unique names prior to this point in the script.


          <PackRotamers name="pack1" task_operations="task1,task2,task3" />


          #因为pack1是PackRotamers的一个实例，在后面我们可以使用直接使用pack1进行操作
</MOVERS>
```

通常而言，即便RS中什么也不写，只有框架RS，rosetta也会将input文件中缺失的氢原子和侧链进行repack排布缺失原子后输出。此外，rosetta还会将标准的rosetta打分表给在输出文件的末尾。





```
<ROSETTASCRIPTS>
    <SCOREFXNS>
        <ScoreFunction name="molmech" weights="mm_std_fa_elec_dslf_fa13" />
        <ScoreFunction name="r15_cart" weights="ref2015" >
            <Reweight scoretype="pro_close" weight="0.0" />
            <Reweight scoretype="cart_bonded" weight="0.625" />
        </ScoreFunction>
    </SCOREFXNS>
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
    </MOVERS>
    <APPLY_TO_POSE>
    </APPLY_TO_POSE>
    <PROTOCOLS>
    </PROTOCOLS>
    <OUTPUT scorefxn="r15_cart" />
</ROSETTASCRIPTS>
```

上面RS示例中的<ScoreFunction/> tag中展示了如何定义不同的能量函数。在这里定义了两个能量函数，第一个能量函数在rosetta里面调用的名字叫mm_std_fa_elec_dslf_fa13，第二个能量函数就是我们常用的ref2015，但这里又对ref2015中的某些能量项进行权重的修改：pro_close设定为0和cart_bonded设定为0.625。（之前我们也介绍过如何使用patch 文件，修改rosetta中内置能量函数的权重）。
注意，虽然molmech能量函数被定义，但是在后面的tag之中从来没有被使用。这在rosetta中也是允许的。修改权重后的ref2015能量函数，在这里被实例化成“r15_cart”，在<output/> tag中被调用，这也就是告诉rosetta去使用实例化后的r15_cart对pose进行打分的意思。
<br/>

***

# 3.控制RS的输出

在RS框架中存在<output/> tag，这个tag可以允许我们像以前使用rosetta的flag文件一样，设置输出选项来控制输出；<SCOREFXNS/> tag允许我们使用其他的一些能量函数。

## 3.1 修改打分函数

**场景一**

```
<ROSETTASCRIPTS>
    <SCOREFXNS>
        <ScoreFunction name="molmech" weights="mm_std_fa_elec_dslf_fa13" />
        <ScoreFunction name="r15_cart" weights="ref2015" >
            <Reweight scoretype="pro_close" weight="0.0" />
            <Reweight scoretype="cart_bonded" weight="0.625" />
        </ScoreFunction>
    </SCOREFXNS>
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
    </MOVERS>
    <APPLY_TO_POSE>
    </APPLY_TO_POSE>
    <PROTOCOLS>
    </PROTOCOLS>
    <OUTPUT scorefxn="r15_cart" />
</ROSETTASCRIPTS>
```

<br/>
上面RS示例中的<ScoreFunction/> tag中展示了如何定义不同的能量函数。在这里定义了两个能量函数，第一个能量函数在rosetta里面调用的名字叫mm_std_fa_elec_dslf_fa13，第二个能量函数就是我们常用的ref2015，但这里又对ref2015中的某些能量项进行权重的修改：pro_close设定为0和cart_bonded设定为0.625。（之前我们也介绍过如何使用patch 文件，修改rosetta中内置能量函数的权重）。
注意，虽然molmech能量函数被定义，但是在后面的tag之中从来没有被使用。这在rosetta中也是允许的。修改权重后的ref2015能量函数，在这里被实例化成“r15_cart”，在<output/> tag中被调用，这也就是告诉rosetta去使用实例化后的r15_cart对pose进行打分的意思。

In [1]:
# 下面我们尝试使用pyrosetta将上面RS示例中的打分函数提取出来，并对结构进行打分你
from pyrosetta.rosetta.protocols.rosetta_scripts import *
from pyrosetta import *
init()

PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python36.ubuntu 2020.28+release.8ecab77aa50ac1301efe53641e07e09ac91fee3b 2020-07-07T16:41:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.ubuntu r260 2020.28+release.8ecab77aa50 8ecab77aa50ac1301efe53641e07e09ac91fee3b http://www.pyrosetta.org 2020-07-07T16:41:06
core.init: command: PyRosetta -ex1 -ex2aro -database /home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=506649970 seed_offset=0 real_seed=506649970
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=506649970 RG_type=mt19937


In [2]:
pose = pose_from_pdb("data/my_ab.pdb")
original_pose = pose.clone()

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 981 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.828125 seconds.
core.import_pose.import_pose: File 'data/my_ab.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 771 845
core.conformation.Conformation: current variant for 771 CYS
core.conformation.Conformation: current variant for 845 CYS
core.conformation.Conformation: current variant for 771 CYD
core.conformation.Conformation: current variant for 845 CYD
core.conformation.Conformation: Found disulfide between residues 891 956
core.conformation.Conformation: current variant for 891 CYS
core.conformation.Conformation: current variant for 956 CYS
core.conformation.Conformation: current variant for 891 CYD
core.conformation.Conformation: current variant for 956 CYD


In [3]:
# 通过Xmlobjects对象读取
with open ("data/Example-ScoreFunction.xml", 'r') as f:
    xml_lines = f.read()

In [4]:
# Pyrosetta直接从string读入会花费较长时间，效率并不高
xml = XmlObjects.create_from_string(xml_lines)

protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="molmech" weights="mm_std_fa_elec_dslf_fa13"/>
		<ScoreFunction name="r15_cart" weights="ref2015">
			<Reweight scoretype="pro_close" weight="0.0"/>
			<Reweight scoretype="cart_bonded" weight="0.625"/>
		</ScoreFunction>
	</SCOREFXNS>
	<RESIDUE_SELECTORS/>
	<TASKOPERATIONS/>
	<FILTERS/>
	<MOVERS/>
	<APPLY_TO_POSE/>
	<PROTOCOLS/>
	<OUTPUT scorefxn="r15_cart"/>
</ROSETTASCRIPTS>
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting

In [5]:
# 以RS定义的名字，获取对应的能量函数对象
molmech_scorefunc = xml.get_score_function("molmech")
r15_cart = xml.get_score_function("r15_cart")

# 查看类型
print(type(molmech_scorefunc))
print(type(r15_cart))

<class 'pyrosetta.rosetta.core.scoring.ScoreFunction'>
<class 'pyrosetta.rosetta.core.scoring.ScoreFunction'>


In [6]:
# 该能量函数对象与前面章节中介绍的能量函数对象别无二致
# 对pose进行打分：
r15_cart(pose)

core.scoring.CartesianBondedEnergy: Creating new peptide-bonded energy container (975)
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibr

16135.774462142144

In [7]:
# 查看打分的详细信息
r15_cart.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000   -5695.317   -5695.317
 fa_rep                       0.550   24719.870   13595.929
 fa_sol                       1.000    3501.170    3501.170
 fa_intra_rep                 0.005    2661.048      13.305
 fa_intra_sol_xover4          1.000     175.028     175.028
 lk_ball_wtd                  1.000     -57.980     -57.980
 fa_elec                      1.000   -1588.297   -1588.297
 hbond_sr_bb                  1.000     -74.000     -74.000
 hbond_lr_bb                  1.000    -511.787    -511.787
 hbond_bb_sc                  1.000    -130.727    -130.727
 hbond_sc                     1.000     -78.766     -78.766
 dslf_fa13                    1.250      -1.572      -1.965
 omega                        0.400     474.068     189.627
 fa_dun 

***

## 3.2 改变构象：RS中定义的Mover

Movers tag是RS的核心部分，Movers指rosetta中一切可以改变构象的操作，包括改变原子坐标、Foldtree、constraint、序列、共价连接性质等。注意也有一些movers不改变原子坐标。


movers的记录可以参考：https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/Movers-RosettaScripts


我们以MinMover为例（https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/MinMover）

该网址中给出了MinMover的定义范本，如下：
```
<MinMover name="(&string;)" jump="(&string;)"
        abs_score_convergence_threshold="(&real;)"
        max_iter="(200 &non_negative_integer;)"
        type="(lbfgs_armijo_nonmonotone &minimizer_type;)"
        tolerance="(0.01 &real;)" cartesian="(false &bool;)"
        bondangle="(0 &bool;)" bondlength="(0 &bool;)" chi="(&bool;)"
        bb="(&bool;)" omega="(true &bool;)"
        bb_task_operations="(&task_operation_comma_separated_list;)"
        chi_task_operations="(&task_operation_comma_separated_list;)"
        bondangle_task_operations="(&task_operation_comma_separated_list;)"
        bondlength_task_operations="(&task_operation_comma_separated_list;)"
        movemap_factory="(&string;)" scorefxn="(&string;)" >
    <MoveMap name="(&string;)" bb="(&bool;)" chi="(&bool;)" jump="(&bool;)" >
        <Jump number="(&non_negative_integer;)" setting="(&bool;)" />
        <Chain number="(&non_negative_integer;)" chi="(&bool;)" bb="(&bool;)" />
        <Span begin="(&non_negative_integer;)" end="(&non_negative_integer;)"
                chi="(&bool;)" bb="(&bool;)" bondangle="(&bool;)" bondlength="(&bool;)" />
        <ResidueSelector selector="(&string;)" chi="(&bool;)" bb="(&bool;)"
                bondangle="(&bool;)" bondlength="(&bool;)" />
    </MoveMap>
</MinMover>
```

<br/>
该MinMover的作用是用来进行侧链或骨架的能量最小化过程（调整二面角）。可以发现在MinMover中亚标签```<MoveMap> </MoveMap>```中可以让我们定义minimization的自由度，如chi，bb等参数设置。一个MinMover中有很多的选项让我们可以精确控制该Mover的行为，这也是RS中强大的一点。注意，大多数选项平常应用的时候本身都存在默认值，不需要修改，除非作为用户我们有修改的理由或明白我们在进行什么操作。


在写mover的时候，用户需要给每个mover都写上一个独特的名字（name关键字），作为其实例化的名字。比如，我可以使用MinMover（相当于一个MinMover类），可以实例化各种不同名字的MinMover，并改变他们的属性，有些我可能想固定BB，有些我想固定CHI等等...可以发现这种类似于类实例化的方法，让我们有更高的自由度去操纵MinMover（其他Mover也是类似的）。

**场景二**

在上面定义了molchem和r15_cart能量函数的基础上，希望定义两个MinMover：第一个命名为min_torsion，其中设定cartesian选项False，将使用默认的扭转角minimization的方法和定义的molchem能量函数；第二个命名为min_cart，需要将cartesian选项打开，并使用r15_cart能量函数。两种Mover都允许BB和CHI的改变。


<br/>
在基于场景一的RS上，我们需要在Movers tag中写入一下内容，进行场景二中的MinMover实例化。

**注意**：
当我们定义了Mover，实际调用使用的时候需要在```<PROTOCOLS></PROTOCOLS>```中申明才可以，以下例子中虽然定义了两个MinMover，但是在Protocol曾中仅使用了名为min_cart的mover。

```
    <MOVERS>    # 实例化两个MinMover，并改变其中的选项（属性）
        <MinMover name="min_torsion" scorefxn="molmech" chi="true" bb="1" cartesian="F" >
        </MinMover>
        <MinMover name="min_cart" scorefxn="r15_cart" chi="true" bb="1" cartesian="T" >
        </MinMover>
    </MOVERS>
    
   ......
   
    <PROTOCOLS>
        <Add mover="min_cart" />    
    </PROTOCOLS>
    <OUTPUT scorefxn="r15_cart" />
```



In [8]:
# 通过Xmlobjects对象读取
with open ("data/Example2-MinMover.xml", 'r') as f2:
    xml2_lines = f2.read()
    
xml2 = XmlObjects.create_from_string(xml2_lines)

protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="molmech" weights="mm_std_fa_elec_dslf_fa13"/>
		<ScoreFunction name="r15_cart" weights="ref2015">
			<Reweight scoretype="pro_close" weight="0.0"/>
			<Reweight scoretype="cart_bonded" weight="0.625"/>
		</ScoreFunction>
	</SCOREFXNS>
	<RESIDUE_SELECTORS/>
	<TASKOPERATIONS/>
	<FILTERS/>
	<MOVERS>
		<MinMover bb="1" cartesian="F" chi="true" name="min_torsion" scorefxn="molmech"/>
		<MinMover bb="1" cartesian="T" chi="true" name="min_cart" scorefxn="r15_cart"/>
	</

In [9]:
# 场景二中使用的RS其实就已经是一个user-defined protocol了
# 我们可以使用以下流程apply到pose上：

# 拷贝一份原始pose到pose2，不操作原有pose
pose2 = pose.clone()

protocol = xml2.get_mover("ParsedProtocol")

protocol.apply(pose2)

# followed by standard pose output procedure
# pose2.dump_pdb("file_name.pdb") with a user-defined file_name
pose2.dump_pdb("./data/my_lab_test.pdb")

protocols.rosetta_scripts.ParsedProtocol: =======================BEGIN MOVER MinMover - min_cart=======================
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 771 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 845 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 891 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 956 BRANCH 1
core.optimization.Minimizer: [ WARNING ] LBFGS MAX CYCLES 200 EXCEEDED, BUT FUNC NOT CONVERGED!
protocols.rosetta_scripts.ParsedProtocol: setting status to success


True

## 3.3 练习

在场景二中定义了两个MinMover，请尝试修改RS文件，让我们可以先进行min_torsion的MinMover（使用molchem能量函数，cartesian=False），再进行min_cart的MinMover（使用r15_cart能量函数，允许cartesian）。注意，到PROTOCOLS层中的任务是按照次序依次进行，每一步的Mover都是从上一步Mover操作后的构象接着操作的。

1. 请写出新的xml格式的RS文件，并将该protocol应用到my_lab.pdb的pose中；
2. 保存自定义RS运行结束后输出的构象PDB文件，并与原文件对比；
3. 观察pyrosetta的输出，你得到了什么信息？
4. 修改```<OUTPUT></OUTPUT>```标签的scorefunction的名字（根据定义的scorefunction），重复上述过程，会发生什么变化？最终构象会改变吗？
